In [23]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
from minepy import MINE
from mlxtend.regressor import StackingRegressor
from collections import defaultdict

In [7]:
data_set = pd.read_csv('../data/train.csv')
test_set = pd.read_csv('../data/test.csv')
Le = LabelEncoder()

y = data_set[['Y']]
data_set = data_set.drop(['Y'], axis=1)
input_data = pd.concat([data_set, test_set], ignore_index=True)

In [8]:
col = pd.DataFrame(input_data.ix[:, 2:].columns, columns=['col'])
TOOL_ID_col_dict = defaultdict(lambda : 0)
for index,row in col.iterrows():
    info=row.col.split('X')[0]
    TOOL_ID_col_dict[info] += 1
    
TOOL_ID_col_dict

In [54]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.mean())      
    for line in data.columns:
        if len(data[line].unique()) == 1:
            data = data.drop([line], axis=1)    
    return data

In [10]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 3 * (Q3 - Q1)
        max_v = Q3 + 3 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].mean()
           
    return data

In [20]:
#标准化
def scala(data):
 #   for col in data.columns:            
 #       data[col] = (data[col] - data[col].mean()) / data[col].std(ddof=0) 
 #   data = data.fillna(0)   
    data = scale(data)
    return data

In [27]:
#PCA
def Pca(data):
    pca = PCA(n_components=0.9)
    pca.fit(data)
    X_new = pca.transform(data) 
    return X_new

In [13]:
#Tsne
def Tsne(data):
    Tsne = TSNE(n_components=2)
    X_new = Tsne.fit_transform(data)
    return X_new

In [14]:
#聚类
def clu(data):
    km = KMeans(n_clusters=3).fit(data)
    clu = pd.DataFrame(km.predict(data), columns=['clu'])
    clu.groupby(['clu']).clu.count()

In [15]:
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

In [16]:
#孤立森林选出异常值过多的特征  暂时没用
def get_outFeature(data):
    clf = IsolationForest(max_samples=20)
    outrate = []
    for col in data.columns: 
        clf.fit(data[[col]])
        y_pred_train = clf.predict(data[[col]])
    
        values = data[[col]].values
        out = pd.DataFrame(values, columns=['columns'])
        out['y'] = y_pred_train
    
        outLine = len(out[out['y'] == -1])
        outRate = outLine / 600
        if outRate > 0.2:
            outrate.append(col)
            
    feature = [x for x in data.columns if x in outrate]
    train = pd.DataFrame(data[feature]).ix[:499, :]
    test = pd.DataFrame(data[feature]).ix[500:, :]
    return train, test

处理X210_

In [37]:
X210_COL = input_data.loc[:, 'TOOL_ID':'210X231']
Le.fit(X210_COL['TOOL_ID'].unique())
X210_COL['TOOL_ID'] = Le.transform(X210_COL['TOOL_ID'])
X210_COL = date_cols(X210_COL)
X210_COL = drop_col(X210_COL)
X210_COL = quantile_dropout(X210_COL)
X210_COL = drop_col(X210_COL)
X210_COL = X210_COL.transpose().drop_duplicates().transpose()
X210_COL = scala(X210_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [38]:
pca_X210 = Pca(X210_COL)
pca_X210 = pd.DataFrame(pca_X210)
pca_X210.reset_index(inplace=True)
pca_X210.rename({'index' : 'ID'}, inplace=True)
pca_X210.drop(['index'], axis=1, inplace=True)

处理X220_

In [39]:
X220_COL = input_data.loc[:, 'Tool':'220X571']
Le.fit(X220_COL['Tool'].unique())
X220_COL['Tool'] = Le.transform(X220_COL['Tool'])
X220_COL = drop_col(X220_COL)
X220_COL = date_cols(X220_COL)
X220_COL = quantile_dropout(X220_COL)
X220_COL = drop_col(X220_COL)
X220_COL = X220_COL.transpose().drop_duplicates().transpose()
X220_COL = scala(X220_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
pca_X220 = Pca(X220_COL)
pca_X220 = pd.DataFrame(pca_X220)
pca_X220.reset_index(inplace=True)
pca_X220.rename({'index' : 'ID'}, inplace=True)
pca_X220.drop(['index'], axis=1, inplace=True)

处理X261_

In [41]:
X261_COL = input_data.loc[:, '261X226':'261X763']
X261_COL = drop_col(X261_COL)
X261_COL = date_cols(X261_COL)
X261_COL = quantile_dropout(X261_COL)
X261_COL = drop_col(X261_COL)
X261_COL = X261_COL.transpose().drop_duplicates().transpose()
X261_COL = scala(X261_COL)

In [42]:
pca_X261 = Pca(X261_COL)
pca_X261 = pd.DataFrame(pca_X261)
pca_X261.reset_index(inplace=True)
pca_X261.rename({'index' : 'ID'}, inplace=True)
pca_X261.drop(['index'], axis=1, inplace=True)

处理X300_   

In [44]:
X300_COL = input_data.loc[:, 'TOOL_ID (#1)':'300X21']
Le.fit(X300_COL['TOOL_ID (#1)'].unique())
X300_COL['TOOL_ID (#1)'] = Le.transform(X300_COL['TOOL_ID (#1)'])
X300_COL = drop_col(X300_COL)
X300_COL = quantile_dropout(X300_COL.ix[:, 1:])
X300_COL = drop_col(X300_COL)
X300_COL = X300_COL.transpose().drop_duplicates().transpose()
X300_COL = scala(X300_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [45]:
pca_X300 = Pca(X300_COL)
pca_X300 = pd.DataFrame(pca_X300)
pca_X300.reset_index(inplace=True)
pca_X300.rename({'index' : 'ID'}, inplace=True)
pca_X300.drop(['index'], axis=1, inplace=True)

处理X310_ 

In [46]:
X310_COL = input_data.loc[:, 'TOOL_ID (#2)':'310X207']
Le.fit(X310_COL['TOOL_ID (#2)'].unique())
X310_COL['TOOL_ID (#2)'] = Le.transform(X310_COL['TOOL_ID (#2)'])
X310_COL = drop_col(X310_COL)
X310_COL = date_cols(X310_COL)
X310_COL = quantile_dropout(X310_COL)
X310_COL = drop_col(X310_COL)
X310_COL = X310_COL.transpose().drop_duplicates().transpose()
X310_COL = scala(X310_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
pca_X310 = Pca(X310_COL)
pca_X310 = pd.DataFrame(pca_X310)
pca_X310.reset_index(inplace=True)
pca_X310.rename({'index' : 'ID'}, inplace=True)
pca_X310.drop(['index'], axis=1, inplace=True)

处理X311_

In [56]:
X311_COL = input_data.loc[:, 'TOOL_ID (#3)':'311X225']
Le.fit(X311_COL['TOOL_ID (#3)'].unique())
X311_COL['TOOL_ID (#3)'] = Le.transform(X311_COL['TOOL_ID (#3)'])
X311_COL = drop_col(X311_COL)
X311_COL = date_cols(X311_COL)
X311_COL = quantile_dropout(X311_COL)
X311_COL = drop_col(X311_COL)
X311_COL = X311_COL.transpose().drop_duplicates().transpose()
X311_COL = scala(X311_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
pca_X311 = Pca(X311_COL)
pca_X311 = pd.DataFrame(pca_X311)
pca_X311.reset_index(inplace=True)
pca_X311.rename({'index' : 'ID'}, inplace=True)
pca_X311.drop(['index'], axis=1, inplace=True)

处理X312_

In [58]:
X312_COL = input_data.loc[:, 'Tool (#1)':'312X798']
Le.fit(X312_COL['Tool (#1)'].unique())
X312_COL['Tool (#1)'] = Le.transform(X312_COL['Tool (#1)'])
X312_COL = drop_col(X312_COL)
X312_COL = date_cols(X312_COL)
X312_COL = quantile_dropout(X312_COL)
X312_COL = drop_col(X312_COL)
X312_COL = X312_COL.transpose().drop_duplicates().transpose()
X312_COL = scala(X312_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\data.py:177: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [59]:
pca_X312 = Pca(X312_COL)
pca_X312 = pd.DataFrame(pca_X312)
pca_X312.reset_index(inplace=True)
pca_X312.rename({'index' : 'ID'}, inplace=True)
pca_X312.drop(['index'], axis=1, inplace=True)

处理X330_

In [60]:
X330_COL = input_data.loc[:, 'Tool (#2)':'330X1311']
Le.fit(X330_COL['Tool (#2)'].unique())
X330_COL['Tool (#2)'] = Le.transform(X330_COL['Tool (#2)'])
X330_COL = drop_col(X330_COL)
X330_COL = date_cols(X330_COL)
X330_COL = quantile_dropout(X330_COL)
X330_COL = drop_col(X330_COL)
X330_COL = X330_COL.transpose().drop_duplicates().transpose()
X330_COL = scala(X330_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [61]:
pca_X330 = Pca(X330_COL)
pca_X330 = pd.DataFrame(pca_X330)
pca_X330.reset_index(inplace=True)
pca_X330.rename({'index' : 'ID'}, inplace=True)
pca_X330.drop(['index'], axis=1, inplace=True)

处理X340_

In [62]:
X340_COL = input_data.loc[:, 'tool':'340X199']
Le.fit(X340_COL['tool'].unique())
X340_COL['tool'] = Le.transform(X340_COL['tool'])
X340_COL = drop_col(X340_COL)
X340_COL = date_cols(X340_COL)
X340_COL = quantile_dropout(X340_COL)
X340_COL = drop_col(X340_COL)
X340_COL = X340_COL.transpose().drop_duplicates().transpose()
X340_COL = scala(X340_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [63]:
pca_X340 = Pca(X340_COL)
pca_X340 = pd.DataFrame(pca_X340)
pca_X340.reset_index(inplace=True)
pca_X340.rename({'index' : 'ID'}, inplace=True)
pca_X340.drop(['index'], axis=1, inplace=True)

处理X344_

In [64]:
X344_COL = input_data.loc[:, 'tool (#1)':'344X398']
Le.fit(X344_COL['tool (#1)'].unique())
X344_COL['tool (#1)'] = Le.transform(X344_COL['tool (#1)'])
X344_COL = drop_col(X344_COL)
X344_COL = date_cols(X344_COL)
X344_COL = quantile_dropout(X344_COL)
X344_COL = drop_col(X344_COL)
X344_COL = X344_COL.transpose().drop_duplicates().transpose()
X344_COL = scala(X344_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [65]:
pca_X344 = Pca(X344_COL)
pca_X344 = pd.DataFrame(pca_X344)
pca_X344.reset_index(inplace=True)
pca_X344.rename({'index' : 'ID'}, inplace=True)
pca_X344.drop(['index'], axis=1, inplace=True)

处理X360_ 

In [66]:
X360_COL = input_data.loc[:, 'TOOL':'360X1452']
Le.fit(X360_COL['TOOL'].unique())
X360_COL['TOOL'] = Le.transform(X360_COL['TOOL'])
X360_COL = drop_col(X360_COL)
X360_COL = date_cols(X360_COL)
X360_COL = quantile_dropout(X360_COL)
X360_COL = drop_col(X360_COL)
X360_COL = X360_COL.transpose().drop_duplicates().transpose()
X360_COL = scala(X360_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [67]:
pca_X360 = Pca(X360_COL)
pca_X360 = pd.DataFrame(pca_X360)
pca_X360.reset_index(inplace=True)
pca_X360.rename({'index' : 'ID'}, inplace=True)
pca_X360.drop(['index'], axis=1, inplace=True)

处理X400

In [68]:
X400_COL = input_data.loc[:, '400X1':'400X230']
X400_COL = drop_col(X400_COL)
X400_COL = date_cols(X400_COL)
X400_COL = quantile_dropout(X400_COL)
X400_COL = drop_col(X400_COL)
X400_COL = X400_COL.transpose().drop_duplicates().transpose()
X400_COL = scala(X400_COL)

In [69]:
pca_X400 = Pca(X400_COL)
pca_X400 = pd.DataFrame(pca_X400)
pca_X400.reset_index(inplace=True)
pca_X400.rename({'index' : 'ID'}, inplace=True)
pca_X400.drop(['index'], axis=1, inplace=True)

处理X420

In [70]:
X420_COL = input_data.loc[:, '420X1':'420X230']
X420_COL = drop_col(X420_COL)
X420_COL = date_cols(X420_COL)
X420_COL = quantile_dropout(X420_COL)

X420_COL = drop_col(X420_COL)
X420_COL = X420_COL.transpose().drop_duplicates().transpose()
X420_COL = scala(X420_COL)

In [71]:
pca_X420 = Pca(X420_COL)
pca_X420 = pd.DataFrame(pca_X420)
pca_X420.reset_index(inplace=True)
pca_X420.rename({'index' : 'ID'}, inplace=True)
pca_X420.drop(['index'], axis=1, inplace=True)

处理X440A

In [72]:
X440A_COL = input_data.loc[:, 'TOOL (#1)':'440AX213']
Le.fit(X440A_COL['TOOL (#1)'].unique())
X440A_COL['TOOL (#1)'] = Le.transform(X440A_COL['TOOL (#1)'])
X440A_COL = drop_col(X440A_COL)
X440A_COL = date_cols(X440A_COL)
X440A_COL = quantile_dropout(X440A_COL)
X440A_COL = drop_col(X440A_COL)
X440A_COL = X440A_COL.transpose().drop_duplicates().transpose()
X440A_COL = scala(X440A_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
pca_X440A = Pca(X440A_COL)
pca_X440A = pd.DataFrame(pca_X440A)
pca_X440A.reset_index(inplace=True)
pca_X440A.rename({'index' : 'ID'}, inplace=True)
pca_X440A.drop(['index'], axis=1, inplace=True)

处理X520

In [74]:
X520_COL = input_data.loc[:, 'Tool (#3)':'520X434']
Le.fit(X520_COL['Tool (#3)'].unique())
X520_COL['Tool (#3)'] = Le.transform(X520_COL['Tool (#3)'])
X520_COL = drop_col(X520_COL)
X520_COL = date_cols(X520_COL)
X520_COL = quantile_dropout(X520_COL)
X520_COL = drop_col(X520_COL)
X520_COL = X520_COL.transpose().drop_duplicates().transpose()
X520_COL = scala(X520_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
pca_X520 = Pca(X520_COL)
pca_X520 = pd.DataFrame(pca_X520)
pca_X520.reset_index(inplace=True)
pca_X520.rename({'index' : 'ID'}, inplace=True)
pca_X520.drop(['index'], axis=1, inplace=True)

处理X750

In [76]:
X750_COL = input_data.loc[:, 'TOOL (#2)':'750X1452']
Le.fit(X750_COL['TOOL (#2)'].unique())
X750_COL['TOOL (#2)'] = Le.transform(X750_COL['TOOL (#2)'])
X750_COL = drop_col(X750_COL)
X750_COL = date_cols(X750_COL)
X750_COL = quantile_dropout(X750_COL)
X750_COL = drop_col(X750_COL)
X750_COL = X750_COL.transpose().drop_duplicates().transpose()
X750_COL = scala(X750_COL)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
pca_X750 = Pca(X750_COL)
pca_X750 = pd.DataFrame(pca_X750)
pca_X750.reset_index(inplace=True)
pca_X750.rename({'index' : 'ID'}, inplace=True)
pca_X750.drop(['index'], axis=1, inplace=True)

In [79]:
data_Set = pd.concat([pca_X210, pca_X220, 
                      pca_X261, pca_X300, 
                      pca_X310, pca_X311,
                      pca_X312, pca_X330, 
                      pca_X340, pca_X344, 
                      pca_X360, pca_X400,
                      pca_X420, pca_X440A, 
                      pca_X520, pca_X750], axis=1)

In [80]:
train = data_set.ix[:499, :]
test = data_set.ix[500:, :]

In [79]:
train.to_csv('train/train.csv', index=False)
test.to_csv('train/test.csv', index=False)
y.to_csv('train/y.csv', index=False)